In [2]:
import csv
import numpy
import networkx as nx
import matplotlib.pyplot as plt
import copy
import time

In [108]:
M_vector=numpy.array([1000,2000,3000,4000,5000])
tran_max=1000
water_filling_packet=5

# Importing Ripple Data 

- Graph
- Transactions

In [5]:
#Importing the set of edges
bi_edges=numpy.loadtxt("GIT files/ripple_graph.txt",dtype=int)
bi_edges=numpy.unique(bi_edges,axis=0)

#Making a graph from the given set of edges
wei=numpy.ones(len(bi_edges)) #initializing the weights of the edges
bi_G = nx.DiGraph() #Initializing the directed graph
for i in range(0,len(bi_edges)):
    bi_G.add_weighted_edges_from([(bi_edges[i,0], bi_edges[i,1],wei[i])]) #adding edges to the graph and assigning weights

#Importing Transactions
with open('GIT files/transactions.csv') as csvDataFile:
    tran=csv.reader(csvDataFile)
    tran=list(tran) #converting the imported data into a list
    tran=numpy.array(tran) #converting the list into numpy array
tran=numpy.ndarray.astype(tran,float)
k=0
for i in range(0,len(tran)):
    if tran[i,2]>tran_max:
        tran=numpy.delete(tran,obj=i,axis=0) #removing all the transactions that are greater than tran_max
    k=k+1
    if len(tran)<k:
        break

# Finding the minimum weighted path in a given graph G from s to d

- Using the Heuristic: Find K- Shortest path using the positive part of the weights and pick the path among these which has the lowest total weight

In [6]:
def minwp_heu(G,s,d,tran,K): 
    X=nx.shortest_simple_paths(G,s,d,'imbalance') 
    #returns a generator which returns one path at a time from shortest to longest
    weight2=100000 #initialize by a very large number
    for j, path in enumerate(X): #enumerate shortest to longest paths 
        weight1=0 #initialize by 0 to calculate the weight of path
        for k in range(0,len(path)-1):
            weight1=weight1+G[path[k]][path[k+1]]['weight']-G[path[k+1]][path[k]]['weight'] 
            #calculating the actual weight of path p
        if min(weight1,weight2)==weight1 and weight1!=weight2: #comparing with the best weight obtained so far
            min_wei_path=path #picking the minimum weight path
            weight2=weight1 #updating the best weight obtained so for
        if j == K-1: #break after looking at K shortest paths according to the imbalance
            break
    var=1
    for l in range(0,len(min_wei_path)-1):
        s=min(G[min_wei_path[l]][min_wei_path[l+1]]['weight'],G[min_wei_path[l+1]][min_wei_path[l]]['weight'],M)
        if G[min_wei_path[l]][min_wei_path[l+1]]['weight']+tran-s>M: 
            #Checking if a has enough capacity to route tran
            var=0
    if var!=0:
        return min_wei_path #We will either get a path from s to d or 0 if min_wei_path cannot route tran
    else:
        return 0

# Routing Tran using path

In [50]:
def UpdateWeights(G,path,tran):
    for i in range(0,len(path)-1):
        s=min(G[path[i]][path[i+1]]['weight'],G[path[i+1]][path[i]]['weight'],M) #calculating the amount of service
        G[path[i]][path[i+1]]['weight']=G[path[i]][path[i+1]]['weight']+tran-s
        G[path[i+1]][path[i]]['weight']=G[path[i+1]][path[i]]['weight']-s
        bi_G[path[i]][path[i+1]]['imbalance']=max(bi_G[path[i]][path[i+1]]['weight']
            -bi_G[path[i+1]][path[i]]['weight'],0)#initializing the weights
        bi_G[path[i+1]][path[i]]['imbalance']=max(bi_G[path[i+1]][path[i]]['weight']
            -bi_G[path[i]][path[i+1]]['weight'],0) #initializing the weights
    return G

# Main Loop

In [278]:
for argument in range(1,len(sys.argv)):
    M=M_vector[int(sys.argv[argument])]
    no_of_succ_tran=0
    amount_of_succ_tran=0
    total_buffer=0
    min_wei_path=0
    for k in range(0,len(bi_edges)):
        bi_G[bi_edges[k,0]][bi_edges[k,1]]['weight']=0.01/len(bi_edges) #initializing the weights
        bi_G[bi_edges[k,0]][bi_edges[k,1]]['imbalance']=0 #initializing the imbalance
    for i in range(0,len(tran)):
        if min_wei_path!=0:
            bi_G=UpdateWeights(bi_G,min_wei_path,0) #Balancing the outstanding balance from prev transaction
            #Ideally, you would clear out the buffer on all the edges but we do it only for the edges for which 
            #there can be some outstanding balance to optimize the run time of the code
        min_wei_path=minwp_heu(bi_G,tran[i,0],tran[i,1],float(tran[i,2]),1) #finding the minimum weighted path from sender to reciever
        if min_wei_path!=0: #If a path exists
            bi_G=UpdateWeights(bi_G,min_wei_path,tran[i,2]) #Route the transaction using min_wei_path
            no_of_succ_tran=no_of_succ_tran+1
            amount_of_succ_tran=amount_of_succ_tran+tran[i,2]
        print(no_of_succ_tran, "out of", i)
    for i in range(0,len(bi_edges)):
        total_buffer=total_buffer+bi_G[bi_edges[i,0]][bi_edges[i,1]]['weight']
    f = open("max_weight_heu.txt","a+")
    f.write('------------------------------------')
    f.write("\r\n")
    f.write("no_of_succ_tran =%f," %no_of_succ_tran)
    f.write("\r\n")
    f.write("amount_of_succ_tran=%f" %amount_of_succ_tran)
    f.write("\r\n")
    f.write("total_buffer = %f," %total_buffer)
    f.write("\r\n")
    f.write("M = %f," %M)
    f.write("\r\n")
    f.write('------------------------------------')
    f.write("\r\n")
    f.close()

1 out of 0
2 out of 1
3 out of 2
4 out of 3
5 out of 4
6 out of 5
7 out of 6
8 out of 7
9 out of 8
10 out of 9
11 out of 10
12 out of 11
13 out of 12
14 out of 13
15 out of 14
16 out of 15
17 out of 16
18 out of 17
19 out of 18
19 out of 19
20 out of 20
21 out of 21
22 out of 22
23 out of 23
24 out of 24
25 out of 25
26 out of 26
27 out of 27
28 out of 28
29 out of 29
30 out of 30
31 out of 31
32 out of 32
33 out of 33
34 out of 34
35 out of 35
36 out of 36
37 out of 37
38 out of 38
39 out of 39
40 out of 40
41 out of 41
42 out of 42
43 out of 43
44 out of 44
45 out of 45
46 out of 46
47 out of 47
48 out of 48
49 out of 49
50 out of 50
51 out of 51
52 out of 52
53 out of 53
54 out of 54
55 out of 55
56 out of 56
57 out of 57
58 out of 58
59 out of 59
60 out of 60
61 out of 61
62 out of 62
63 out of 63
63 out of 64
64 out of 65
65 out of 66
66 out of 67
67 out of 68
68 out of 69
69 out of 70
70 out of 71
71 out of 72
72 out of 73
73 out of 74
74 out of 75
75 out of 76
76 out of 77
77 ou

# Water Filling
- Using minwp_heu to find the shortest weighted path but divide the transaction by water_filling _packet and route each one of them seperately
- UpdateWeights is replaced by AddWeights and Balancing as we first AddWeights to all the multiple paths and then balance all the outstanding balance together

In [116]:
def AddWeights(G,path,tran,d):
    for i in range(0,len(path)-1):
        if path[i]!=d:
            G[path[i]][path[i+1]]['weight']=G[path[i]][path[i+1]]['weight']+tran
            bi_G[path[i]][path[i+1]]['imbalance']=max(bi_G[path[i]][path[i+1]]['weight']
                -bi_G[path[i+1]][path[i]]['weight'],0)
            bi_G[path[i+1]][path[i]]['imbalance']=max(bi_G[path[i+1]][path[i]]['weight']
                -bi_G[path[i]][path[i+1]]['weight'],0) 
    return G

In [118]:
def Balancing(G,path,d):
    for i in range(0,len(path)-1):
        if path[i]!=d:
            s=min(G[path[i]][path[i+1]]['weight'],G[path[i+1]][path[i]]['weight'],M) #calculating the amount of service
            G[path[i]][path[i+1]]['weight']=G[path[i]][path[i+1]]['weight']-s
            G[path[i+1]][path[i]]['weight']=G[path[i+1]][path[i]]['weight']-s
            bi_G[path[i]][path[i+1]]['imbalance']=max(bi_G[path[i]][path[i+1]]['weight']
                -bi_G[path[i+1]][path[i]]['weight'],0)
            bi_G[path[i+1]][path[i]]['imbalance']=max(bi_G[path[i+1]][path[i]]['weight']
                -bi_G[path[i]][path[i+1]]['weight'],0) 
    return G

# Water Filling

In [130]:
for argument in range(1,len(sys.argv)):
    M=M_vector[int(sys.argv[argument])]
    no_of_succ_tran=0
    amount_of_succ_tran=0
    total_buffer=0
    min_wei_path=0
    for k in range(0,len(bi_edges)):
        bi_G[bi_edges[k,0]][bi_edges[k,1]]['weight']=0 #initializing the weights
        bi_G[bi_edges[k,0]][bi_edges[k,1]]['imbalance']=0 #initializing the imbalance
    for i in range(0,len(tran)):
        path_list=[] #to consolidate all the paths chosen to route a transaction
        min_wei_path=0
        var=1
        transaction=tran[i,2]/water_filling_packet #dividing the transaction into water_filling_packet number of packets
        for j in range(0,water_filling_packet):
            min_wei_path=minwp_heu(bi_G,tran[i,0],tran[i,1],transaction,1) 
            #finding the minimum weighted path from sender to reciever
            if min_wei_path!=0: #If a path exists
                bi_G=AddWeights(bi_G,min_wei_path,transaction,tran[i,1]) #Add the transaction to min_wei_path
                path_list.extend(min_wei_path) #add the path to the path list
            else:
                var=0 #even if partial transaction fails, abort the whole transaction
                if path_list!=[]:
                    bi_G=AddWeights(bi_G,path_list,-transaction,tran[i,1]) 
                    #remove the transaction that was added to the graph
                break
        if var!=0: #if the whole transaction went through
            bi_G=Balancing(bi_G,path_list,tran[i,1]) #balance the outstanding transaction
            no_of_succ_tran=no_of_succ_tran+1 #update the number of successful transactions
            amount_of_succ_tran=amount_of_succ_tran+tran[i,2] #update the amount of successful transactions
    for i in range(0,len(bi_edges)):
        total_buffer=total_buffer+bi_G[bi_edges[i,0]][bi_edges[i,1]]['weight'] 
        #output the total imbalance in the network
    f = open("Blockchain/water_filling.txt","a+")
    f.write('------------------------------------')
    f.write("\r\n")
    f.write("no_of_succ_tran =%f," %no_of_succ_tran)
    f.write("\r\n")
    f.write("amount_of_succ_tran=%f" %amount_of_succ_tran)
    f.write("\r\n")
    f.write("total_buffer = %f," %total_buffer)
    f.write("\r\n")
    f.write("M = %f," %M)
    f.write("\r\n")
    f.write('------------------------------------')
    f.write("\r\n")
    f.close()

weight ( 1 2 ) 0.0
weight ( 1 5 ) 0.0
weight ( 2 3 ) 0.8
weight ( 2 4 ) 0.8
weight ( 3 4 ) 0.8
weight ( 4 5 ) 1.6
weight ( 2 1 ) 2.4
weight ( 5 1 ) 1.6
weight ( 3 2 ) 0.0
weight ( 4 2 ) 0.0
weight ( 4 3 ) 0.0
weight ( 5 4 ) 0.0
imbalance ( 1 2 ) 0
imbalance ( 1 5 ) 0
imbalance ( 2 3 ) 0.8
imbalance ( 2 4 ) 0.8
imbalance ( 3 4 ) 0.8
imbalance ( 4 5 ) 1.6
imbalance ( 2 1 ) 2.4
imbalance ( 5 1 ) 1.6
imbalance ( 3 2 ) 0
imbalance ( 4 2 ) 0
imbalance ( 4 3 ) 0
imbalance ( 5 4 ) 0
0
weight ( 1 2 ) 0.0
weight ( 1 5 ) 0.0
weight ( 2 3 ) 0.8
weight ( 2 4 ) 0.8
weight ( 3 4 ) 0.8
weight ( 4 5 ) 1.6
weight ( 2 1 ) 4.4
weight ( 5 1 ) 1.6
weight ( 3 2 ) 0.0
weight ( 4 2 ) 0.0
weight ( 4 3 ) 0.0
weight ( 5 4 ) 0.0
imbalance ( 1 2 ) 0
imbalance ( 1 5 ) 0
imbalance ( 2 3 ) 0.8
imbalance ( 2 4 ) 0.8
imbalance ( 3 4 ) 0.8
imbalance ( 4 5 ) 1.6
imbalance ( 2 1 ) 4.4
imbalance ( 5 1 ) 1.6
imbalance ( 3 2 ) 0
imbalance ( 4 2 ) 0
imbalance ( 4 3 ) 0
imbalance ( 5 4 ) 0
[2, 1]
weight ( 1 2 ) 0.0
weight ( 1 5

# Test Cases

In [98]:
M=10
bi_edges=numpy.array([[1,2],[1,5],[2,3],[2,4],[3,4],[4,5],[2,1],[5,1],[3,2],[4,2],[4,3],[5,4]])
#wei=1
#bi_G = nx.DiGraph() #Initializing the directed graph
#for i in range(0,len(bi_edges)):
#    bi_G[bi_edges[i,0]][bi_edges[i,1]]['imbalance']=0
#    bi_G[bi_edges[i,0]][bi_edges[i,1]]['weight']=0
#bi_G=UpdateWeights(bi_G,[2,1],4)
print(minwp_heu(bi_G,1,4,1,10)) 
for i in range(0,len(bi_edges)):
    print("weight","(",bi_edges[i,0],bi_edges[i,1],")",bi_G[bi_edges[i,0]][bi_edges[i,1]]['weight'])
for i in range(0,len(bi_edges)):
    print("imbalance","(",bi_edges[i,0],bi_edges[i,1],")",bi_G[bi_edges[i,0]][bi_edges[i,1]]['imbalance'])

[1, 2, 3, 4]
weight ( 1 2 ) 9
weight ( 1 5 ) 0
weight ( 2 3 ) 0
weight ( 2 4 ) 0
weight ( 3 4 ) 0
weight ( 4 5 ) 0
weight ( 2 1 ) 4
weight ( 5 1 ) 0
weight ( 3 2 ) 3
weight ( 4 2 ) 1
weight ( 4 3 ) 5
weight ( 5 4 ) 1
imbalance ( 1 2 ) 5
imbalance ( 1 5 ) 0
imbalance ( 2 3 ) 0
imbalance ( 2 4 ) 0
imbalance ( 3 4 ) 0
imbalance ( 4 5 ) 0
imbalance ( 2 1 ) 0
imbalance ( 5 1 ) 0
imbalance ( 3 2 ) 3
imbalance ( 4 2 ) 1
imbalance ( 4 3 ) 5
imbalance ( 5 4 ) 1
